### Crater Rim Erasure

This notebook visualizes crater rim erasure as a result of new crater formation. A simulation is run and craters are plotted with erased crater segments shown in blue.

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools

from typing import Dict

from saturation.simulation import *
from saturation.distributions import *
from saturation.geometry import *
from saturation.plotting import *
from saturation.crater_record import *

These first three plots show an effective radius multiplier of 1 after 100, 200, and 300 craters.

In [2]:
x_min = 12
x_max = 250
r_stat = 1 * x_min
min_rim_percentage = 0.5
cdf_slope = 1.5
observed_terrain_size = 1000
terrain_padding = 125
full_terrain_size = observed_terrain_size + 2 * terrain_padding
n_craters = 150
effective_radius_multiplier = 1

np.random.seed(123)
distribution = ParetoProbabilityDistribution(cdf_slope=cdf_slope, x_min=x_min, x_max=x_max)
craters = list(itertools.islice(get_craters(distribution, full_terrain_size), n_craters))

In [ ]:
crater_record = CraterRecord(r_stat, min_rim_percentage, effective_radius_multiplier, observed_terrain_size, terrain_padding)

for crater in craters[:50]:
    crater_record.add(crater)
plot_crater_record(crater_record, observed_terrain_size, terrain_padding, figsize=6)

for crater in craters[50:100]:
    crater_record.add(crater)
plot_crater_record(crater_record, observed_terrain_size, terrain_padding, figsize=6)

for crater in craters[100:]:
    crater_record.add(crater)
plot_crater_record(crater_record, observed_terrain_size, terrain_padding, figsize=6)

These plots show an effective radius multiplier of 1.5

In [ ]:
crater_record = CraterRecord(r_stat, min_rim_percentage, 1.5, observed_terrain_size, terrain_padding)

for crater in craters[:50]:
    crater_record.add(crater)
plot_crater_record(crater_record, observed_terrain_size, terrain_padding, figsize=6)

for crater in craters[50:100]:
    crater_record.add(crater)
plot_crater_record(crater_record, observed_terrain_size, terrain_padding, figsize=6)

for crater in craters[100:]:
    crater_record.add(crater)
plot_crater_record(crater_record, observed_terrain_size, terrain_padding, figsize=6)

### Examining Arcs

In [ ]:
r_stat = 1
min_rim_percentage = 0.5
observed_terrain_size = 1000
terrain_padding = 0
full_terrain_size = observed_terrain_size + 2 * terrain_padding
effective_radius_multiplier = 1

craters = [
    Crater(id=1, x=500, y=500, radius=300),
    Crater(id=2, x=850, y=550, radius=150),
    Crater(id=3, x=250, y=450, radius=150),
    Crater(id=4, x=280, y=450, radius=100),
    Crater(id=5, x=500, y=800, radius=100),
    Crater(id=6, x=500, y=300, radius=250),
]
crater_record = CraterRecord(r_stat, min_rim_percentage, effective_radius_multiplier, observed_terrain_size, terrain_padding)

In [ ]:
for crater in craters:
    crater_record.add(crater)
plot_crater_record(crater_record, observed_terrain_size, terrain_padding, figsize=6)

In [ ]:
# Arc remaining
1 - sum(x[1] - x[0] for x in crater_record._erased_arcs[1]) / (2 * np.pi)

In [ ]:
# Compare with plot exported to an image
fig, ax = plt.subplots(figsize=(1, 1), dpi=observed_terrain_size)
fig.tight_layout()
plt.tight_layout()

ax.set_frame_on(False)
ax.axis('off')
ax.set_xlim([terrain_padding, observed_terrain_size + terrain_padding])
ax.set_ylim([terrain_padding, observed_terrain_size + terrain_padding])

plt.subplots_adjust(left=0.0,
                    bottom=0.0, 
                    right=1, 
                    top=1, 
                    wspace=0.0, 
                    hspace=0.0)

# Plot craters
first = craters[0]
plot_circle((first.x, first.y), first.radius, ax, fill=False, color='black', antialiased=False, lw=1/observed_terrain_size*72)
   
img = convert_plot_to_array(fig, show_plot=True)
before = img.sum()

# After
for crater in craters[1:]:
    plot_circle((crater.x, crater.y), crater.radius, ax, fill=True, color='white', antialiased=False, lw=0)
    
img = convert_plot_to_array(fig, show_plot=True)
after = img.sum()

after/before

In [3]:
craters = list(itertools.islice(get_craters(distribution, full_terrain_size), 1500))

In [17]:
new_crater = Crater(id=2000, x=500, y=500, radius=50)
effective_radius = new_crater.radius * effective_radius_multiplier

In [24]:
%%timeit

for x in range(1000):
    craters_in_range = [
        old_crater.id
        for old_crater in craters
        if old_crater.radius - effective_radius < np.sqrt((old_crater.x - new_crater.x) ** 2 + (old_crater.y - new_crater.y) ** 2) < old_crater.radius + effective_radius
    ]

3.48 s ± 169 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
# As dataframe
data = pd.DataFrame(craters).set_index(['id'])

In [41]:
%%timeit
for x in range(1000):
    distances = np.sqrt((data.x - new_crater.x) ** 2 + (data.y - new_crater.y) ** 2)
    result = data[(distances < data.radius + effective_radius) & (distances > data.radius - effective_radius)]

569 ms ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
distances = np.sqrt((data.x - new_crater.x) ** 2 + (data.y - new_crater.y) ** 2)
data.index[(distances < data.radius + effective_radius) & (distances > data.radius - effective_radius)]

Int64Index([48, 134, 192, 227, 719, 921, 1044, 1137, 1190, 1328, 1357, 1379,
            1454],
           dtype='int64', name='id')

In [38]:
pd.concat([data, pd.DataFrame([new_crater]).set_index(['id'])], axis=0)

,x,y,radius
id,,,
1,116.783836,1046.832636,16.981127
2,827.145675,1179.000698,14.441522
3,16.449789,30.185507,26.892517
4,1155.689856,584.162841,16.348949
5,678.575531,1073.646047,23.948054
...,...,...,...
1497,1063.357980,922.748960,17.574876
1498,971.011412,187.231537,14.010172
1499,736.100687,130.497189,24.775013


In [39]:
data.drop([1], axis=0)

,x,y,radius
id,,,
2,827.145675,1179.000698,14.441522
3,16.449789,30.185507,26.892517
4,1155.689856,584.162841,16.348949
5,678.575531,1073.646047,23.948054
6,291.224871,968.225256,13.199818
...,...,...,...
1496,425.014892,201.274101,13.593734
1497,1063.357980,922.748960,17.574876
1498,971.011412,187.231537,14.010172
